### Import libraries

In [1]:
# Import Libraries
import pandas as pd
import numpy as np
from dash import Dash, dcc, html, Input, Output
import plotly.express as px
from sklearn.linear_model import LinearRegression
import socket

### Customize

In [2]:
# Customize paramaters
my_input_file='0409 my_input.csv'
my_host = '127.0.0.4'
my_features = ['Customer_Satisfaction', 'Service_Efficiency', 'Employee_Training_Hours', 'Innovation_Score']
my_target = 'Net_Profit_Margin'

### Load data and train model

In [3]:
# Read data
data = pd.read_csv(my_input_file)

# ML Model to Predict Financial Outcome

X = data[my_features]
y = data[my_target]
model = LinearRegression().fit(X, y)
data['Predicted_Profit_Margin'] = model.predict(X)

### Setup dashboard app

In [4]:
# Dash App Setup

app = Dash(__name__)

perspective_map = {
    'Financial': ['Revenue_Growth', 'Net_Profit_Margin', 'Predicted_Profit_Margin'],
    'Customer': ['Customer_Satisfaction'],
    'Internal Process': ['Service_Efficiency'],
    'Learning & Growth': ['Employee_Training_Hours', 'Innovation_Score']
}

app.layout = html.Div([
    html.H2("Balanced Scorecard Dashboard (Financial & Non-Financial KPIs)"),

    html.Label("Select BSC Perspective:"),
    dcc.Dropdown(
        id='perspective_selector',
        options=[{'label': k, 'value': k} for k in perspective_map],
        value='Financial',
        clearable=False
    ),

    dcc.Graph(id='bsc_kpi_graph')
])


### Display perspectives

In [5]:

# Callback to Display Selected Perspective 
@app.callback(
    Output('bsc_kpi_graph', 'figure'),
    Input('perspective_selector', 'value')
)
def update_chart(perspective):
    selected_kpis = perspective_map[perspective]
    df_melt = data[['Branch'] + selected_kpis].melt(id_vars='Branch', var_name='KPI', value_name='Value')

    fig = px.bar(df_melt, x='Branch', y='Value', color='KPI',
                 barmode='group', title=f'{perspective} Perspective KPIs')
    fig.update_layout(xaxis_tickangle=-45)
    return fig

# Display Dash server address
ip_address = socket.gethostbyname(socket.gethostname())
print(f"Dash app running at http://{ip_address}:8050")

# Run the App
if __name__ == '__main__':

    # For external access prefix the first option  with # and remove the # from the second option
    app.run(my_host, port=8050, debug=True)
    #app.run(host='0.0.0.0', port=8050, mode='external', debug=True))

Dash app running at http://192.168.1.73:8050
